# CNN Baseline Code

## 라이브러리 호출

In [1]:
import os
import tensorflow as tf
import keras

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.inception_v3 import InceptionV3

from tensorflow.keras.layers import GlobalAveragePooling2D, MaxPooling2D
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization

from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
from keras.callbacks import ReduceLROnPlateau


from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [2]:
tf.random.set_seed(42)

## 데이터 불러오기

In [3]:
PATH = '/Users/hanhyeongu/Desktop/codestates/CP1/Korea_Food_Image/'
image_size = 150

original_datagen = ImageDataGenerator(rescale=1./255)

train_generator = original_datagen.flow_from_directory(
        PATH+'train',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')

val_generator = original_datagen.flow_from_directory(
        PATH+'validation',
        target_size=(image_size, image_size),
        batch_size=32,
        class_mode='categorical')

Found 103131 images belonging to 129 classes.
Found 26209 images belonging to 129 classes.


In [4]:
class_names = train_generator.class_indices
class_names = {value:key for key, value in class_names.items()}
class_names

{0: '간장게장',
 1: '갈비구이',
 2: '갈비찜',
 3: '갈비탕',
 4: '갈치구이',
 5: '갈치조림',
 6: '감자전',
 7: '감자조림',
 8: '감자탕',
 9: '갓김치',
 10: '건새우볶음',
 11: '경단',
 12: '계란말이',
 13: '계란찜',
 14: '고등어구이',
 15: '고등어조림',
 16: '고사리나물',
 17: '고추튀김',
 18: '곱창구이',
 19: '곱창전골',
 20: '김밥',
 21: '김치볶음밥',
 22: '김치전',
 23: '김치찌개',
 24: '깍두기',
 25: '깻잎장아찌',
 26: '꼬막찜',
 27: '꽁치조림',
 28: '꿀떡',
 29: '나박김치',
 30: '닭갈비',
 31: '닭볶음탕',
 32: '더덕구이',
 33: '도토리묵',
 34: '동그랑땡',
 35: '동태찌개',
 36: '된장찌개',
 37: '두부김치',
 38: '두부조림',
 39: '땅콩조림',
 40: '떡갈비',
 41: '떡볶이',
 42: '라면',
 43: '라볶이',
 44: '막국수',
 45: '만두',
 46: '매운탕',
 47: '멍게',
 48: '메추리알장조림',
 49: '멸치볶음',
 50: '무국',
 51: '무생채',
 52: '물냉면',
 53: '물회',
 54: '미역국',
 55: '미역줄기볶ᄋ

In [5]:
## 적절한 batch size, epoch 수 생각
num_classes = train_generator.num_classes

# Model 

In [6]:
def densenet():
  densenet = DenseNet201(
    include_top=False,
    input_shape=(image_size,image_size,3),
    weights='imagenet'
    )
  
  
  densenet = Sequential([
      densenet,
      Flatten(), 
      Dense(256, activation='relu', kernel_regularizer='l2'),
      BatchNormalization(),
      Dropout(0.2),
      Dense(num_classes, activation='softmax')
  ])

  base_learning_rate = 0.0001

  densenet.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
  
  return densenet

In [7]:
## callback & EarlyStopping

epoch = 5

cur_dir = '/Users/hanhyeongu/Desktop/codestates/CP1/Yam-Pick_DS/callbacks' # callback 데이터 저장 
ckpt_path = os.path.join(cur_dir, 'callback_ckpt')
os.makedirs(ckpt_path, exist_ok=True)

ckpt_filepath = os.path.join(ckpt_path, 'DenseNet_model_{epoch:04d}-{val_loss:.3f}-{val_accuracy:.3f}.h5')

# 특정 기준일 경우에만 저장
# 아래 코드는 val_loss가 기존 최고 성능 모델에 비해 낮아질 때만 저장
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_filepath, 
                                                   monitor='val_loss', 
                                                   save_best_only=True,
                                                   save_weights_only=False) # save_weights_only=False 모델 전체 저장

# # 모든 epoch 결과 저장
# ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath = ckpt_filepath,
#     # save_best_only = False, 
#     save_weights_only = True, # False로 설정할 경우 모델 전체 저장
#     save_freq = 'epoch')


# loss 가 3번 동안 나아지지 않으면 훈련 종료, val_loss, val_accuracy 사용 가능
early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2) 

inception_ml = densenet()
inception_hist = inception_ml.fit_generator(train_generator,
          validation_data=val_generator,
          epochs=epoch,
          callbacks=[ckpt_callback, early_stop_callback])

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

74836368/74836368 [==============================] - 3s 0us/step
Epoch 1/5


/var/folders/tb/x7hcv5j94gn9b46c7qc1vfth0000gn/T/ipykernel_35365/2924118385.py:30: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  inception_hist = inception_ml.fit_generator(train_generator,


  50/3223 [..............................] - ETA: 33:26 - loss: 8.6147 - accuracy: 0.0481

/Users/hanhyeongu/miniforge3/envs/deep/lib/python3.8/site-packages/PIL/Image.py:992: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


2055/3223 [==================>...........] - ETA: 11:03 - loss: 3.1483 - accuracy: 0.5323

/Users/hanhyeongu/miniforge3/envs/deep/lib/python3.8/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Truncated File Read
  warnings.warn(str(msg))


3223/3223 [==============================] - 2051s 609ms/step - loss: 2.6510 - accuracy: 0.5792 - val_loss: 1.6105 - val_accuracy: 0.6784
Epoch 2/5
3223/3223 [==============================] - 1898s 588ms/step - loss: 1.2411 - accuracy: 0.7532 - val_loss: 1.3523 - val_accuracy: 0.7281
Epoch 3/5
3223/3223 [==============================] - 1892s 587ms/step - loss: 0.9054 - accuracy: 0.8255 - val_loss: 1.3932 - val_accuracy: 0.7233
Epoch 4/5
3223/3223 [==============================] - 1903s 590ms/step - loss: 0.7066 - accuracy: 0.8743 - val_loss: 1.2776 - val_accuracy: 0.7579
Epoch 5/5
3223/3223 [==============================] - 1966s 610ms/step - loss: 0.5835 - accuracy: 0.9026 - val_loss: 1.3192 - val_accuracy: 0.7602


이후 모델 학습은 너무 많은 시간이 소요되어 패스..!  

최고 성능을 보였던 epoch=4모델 선택

In [12]:
# callback 결과 불러오기

# 가중치만 저장
checkpoint_path = '/Users/hanhyeongu/Desktop/codestates/CP1/Yam-Pick_DS/callbacks/callback_ckpt/DenseNet_model_0004-1.278-0.758.h5'
model = densenet()
model.load_weights(checkpoint_path)
model.evaluate_generator(val_generator)

# model.fit_generator(train_generator,
#                     validation_data=val_generator,
#                     epochs=2,
#                     callbacks=[ckpt_callback, early_stop_callback]) 

In [11]:
import pickle
def model_save_file(model, file_name):
    model.save(file_name+'.h5')
    with open(file_name+"_History", 'wb') as file: # 히스토리 저장
        pickle.dump(model.history, file)

model_name = '/Users/hanhyeongu/Desktop/codestates/CP1/Yam-Pick_DS/Model_Save/DenseNet201'
model_save_file(model, model_name)